In [1]:
# %%
import os
import numpy as np
from functools import partial
import math
from tqdm import tqdm
import time as time
import torch

os.environ["CUDA_VISIBLE_DEVICES"]="1"

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))


from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from torchmetrics.functional import structural_similarity_index_measure 
from torchmetrics.functional import peak_signal_noise_ratio 

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import scipy.io as sio
from astropy.io import fits

import large_scale_UQ as luq
from large_scale_UQ.utils import to_numpy, to_tensor
from convex_reg import utils as utils_cvx_reg


import glob


True
1
0
NVIDIA A100-PCIE-40GB
Using device: cuda


In [2]:

result_dir = '/disk/xray0/tl3/repos/large-scale-UQ/debug/sampling-outputs/'

file_list = []

for file in glob.glob(result_dir+'ULA_CRR_*.npy'):
    # file_list.append(file)
    print('Processing: ', file)
    results = np.load(file, allow_pickle=True)[()]

    frac_delta = results['params']['frac_delta']
    lmbd = results['params']['lmbd']
    mu = results['params']['mu']
    n_samples = results['params']['n_samples']
    frac_delta = results['params']['frac_delta']
    frac_delta = results['params']['frac_delta']
    thinning = np.int64(1e3)
    frac_burnin = 0.2

    cmap = 'cubehelix'
    save_dir = '/disk/xray0/tl3/repos/large-scale-UQ/debug/sampling-outputs/'

    savefig_dir = save_dir + 'figs/'

    superpix_sizes = [32,16,8,4,1]
    alpha_prob = 0.01

    means_list = results['means_list']
    quantiles = results['quantiles']
    st_dev_down = results['st_dev_down']

    # Define prefix
    save_prefix = 'ULA_CRR_frac_delta_{:.1e}_lmbd_{:.1e}_mu_{:.1e}_nsamples_{:.1e}_thinning_{:.1e}_frac_burn_{:.1e}'.format(
        frac_delta, lmbd, mu, n_samples, thinning, frac_burnin
    )

    for it, pix_size in enumerate(superpix_sizes):

        fig = plt.figure(figsize=(20,5))

        plt.subplot(131)
        ax = plt.gca()
        ax.set_title(f'Mean value, <Mean val>={np.mean(means_list[it]):.2e} pix size={pix_size:d}')
        im = ax.imshow(means_list[it], cmap=cmap)
        divider = make_axes_locatable(ax)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im, cax=cax, orientation='vertical')
        ax.set_yticks([]);ax.set_xticks([])

        plt.subplot(132)
        ax = plt.gca()
        ax.set_title(f'St Dev, <St Dev>={np.mean(st_dev_down[it]):.2e} pix size={pix_size:d}')
        im = ax.imshow(st_dev_down[it], cmap=cmap)
        divider = make_axes_locatable(ax)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im, cax=cax, orientation='vertical')
        ax.set_yticks([]);ax.set_xticks([])

        plt.subplot(133)
        LCI = quantiles[it][1,:,:] - quantiles[it][0,:,:]
        ax = plt.gca()
        ax.set_title(f'LCI, <LCI>={np.mean(LCI):.2e} pix size={pix_size:d}')
        im = ax.imshow(LCI, cmap=cmap)
        divider = make_axes_locatable(ax)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im, cax=cax, orientation='vertical')
        ax.set_yticks([]);ax.set_xticks([])
        plt.savefig(savefig_dir+save_prefix+'_UQ_pixel_size_{:d}.pdf'.format(pix_size))
        plt.close()
        

    for it, pix_size in enumerate(superpix_sizes):

        fig = plt.figure(figsize=(20,5))

        plt.subplot(131)
        ax = plt.gca()
        ax.set_title(f'Mean value, <Mean val>={np.mean(means_list[it]):.2e} pix size={pix_size:d}')
        im = ax.imshow(means_list[it], cmap=cmap)
        divider = make_axes_locatable(ax)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im, cax=cax, orientation='vertical')
        ax.set_yticks([]);ax.set_xticks([])

        plt.subplot(132)
        ax = plt.gca()
        ax.set_title(f'St Dev, <St Dev>={np.mean(st_dev_down[it]):.2e} pix size={pix_size:d}')
        im = ax.imshow(st_dev_down[it], cmap=cmap)
        divider = make_axes_locatable(ax)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im, cax=cax, orientation='vertical')
        ax.set_yticks([]);ax.set_xticks([])

        plt.subplot(133)
        LCI = quantiles[it][1,:,:] - quantiles[it][0,:,:]
        ax = plt.gca()
        ax.set_title(f'LCI, <LCI>={np.mean(LCI):.2e} pix size={pix_size:d}')
        im = ax.imshow(LCI, cmap=cmap)
        divider = make_axes_locatable(ax)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im, cax=cax, orientation='vertical')
        ax.set_yticks([]);ax.set_xticks([])
        plt.savefig(savefig_dir+save_prefix+'_UQ_pixel_size_{:d}_alpha_prob_0.05.pdf'.format(pix_size))
        plt.close()


Processing:  /disk/xray0/tl3/repos/large-scale-UQ/debug/sampling-outputs/ULA_CRR_frac_delta_5.0e-01_lmbd_2.0e+04_mu_2.0e+01_nsamples_1.0e+03_thinning_1.0e+03_frac_burn_2.0e-01_vars.npy
Processing:  /disk/xray0/tl3/repos/large-scale-UQ/debug/sampling-outputs/ULA_CRR_frac_delta_2.0e-01_lmbd_2.0e+04_mu_2.0e+01_nsamples_1.0e+03_thinning_1.0e+03_frac_burn_2.0e-01_vars.npy
Processing:  /disk/xray0/tl3/repos/large-scale-UQ/debug/sampling-outputs/ULA_CRR_frac_delta_1.0e-01_lmbd_1.0e+04_mu_2.0e+01_nsamples_1.0e+03_thinning_1.0e+03_frac_burn_2.0e-01_vars.npy
Processing:  /disk/xray0/tl3/repos/large-scale-UQ/debug/sampling-outputs/ULA_CRR_frac_delta_1.0e-01_lmbd_2.0e+04_mu_2.0e+01_nsamples_1.0e+03_thinning_1.0e+03_frac_burn_2.0e-01_vars.npy
Processing:  /disk/xray0/tl3/repos/large-scale-UQ/debug/sampling-outputs/ULA_CRR_frac_delta_2.0e-01_lmbd_1.0e+04_mu_2.0e+01_nsamples_1.0e+03_thinning_1.0e+03_frac_burn_2.0e-01_vars.npy
Processing:  /disk/xray0/tl3/repos/large-scale-UQ/debug/sampling-outputs/UL